In [ ]:
#https://www.youtube.com/watch?v=wp9BudYGZyA&t=2s

In [ ]:
#this notebook is just the custom data training on utkarsh data ,but the accuracy is bad ,it gives a idea on training of bert multiclassifier

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/atr_july_sort_gds.csv')
df.head()

,Unnamed: 0,Merchant Type Code,Merchant Type Desc,Merchant Location,Merchant Type,Tran Type Desc
0,3,5813,Drinking Places,RAMJI RESTAURANT THANE MAIN,POS,Goods and services
1,46,6540,NaN,OneMobiKwikSystemPLtd Gurgaon hrIN,ECom,Goods and services
2,57,5541,Service Stations,B P BILASPUR GURGAON HRIN,POS,Goods and services
3,64,5541,Service Stations,MS STANDARD SERVICE STAMuzaffarpur BRIN,POS,Goods and services
4,65,6300,Insurance Underwriting Premiums,FreechargePaymentTechnoMumbai mhIN,ECom,Goods and services


In [ ]:
df['encoded_cat'] = df['Merchant Type Code'].astype('category').cat.codes

In [ ]:
df.dtypes

Unnamed: 0             int64
Merchant Type Code     int64
Merchant Type Desc    object
Merchant Location     object
Merchant Type         object
Tran Type Desc        object
encoded_cat            int16
dtype: object

In [ ]:
#df.drop(118202, inplace=True)
df['encoded_cat'] = df['encoded_cat'].astype(int)

In [ ]:
df.head()

,Unnamed: 0,Merchant Type Code,Merchant Type Desc,Merchant Location,Merchant Type,Tran Type Desc,encoded_cat
0,3,5813,Drinking Places,RAMJI RESTAURANT THANE MAIN,POS,Goods and services,99
1,46,6540,NaN,OneMobiKwikSystemPLtd Gurgaon hrIN,ECom,Goods and services,138
2,57,5541,Service Stations,B P BILASPUR GURGAON HRIN,POS,Goods and services,76
3,64,5541,Service Stations,MS STANDARD SERVICE STAMuzaffarpur BRIN,POS,Goods and services,76
4,65,6300,Insurance Underwriting Premiums,FreechargePaymentTechnoMumbai mhIN,ECom,Goods and services,136


In [ ]:
df.dtypes

Unnamed: 0             int64
Merchant Type Code     int64
Merchant Type Desc    object
Merchant Location     object
Merchant Type         object
Tran Type Desc        object
encoded_cat            int64
dtype: object

In [ ]:
# from google.colab import files
# df.to_csv('output.csv', encoding = 'utf-8-sig') 
# files.download('output.csv')

In [ ]:
num_label=max(df['encoded_cat'].unique())+1

In [ ]:
df['encoded_cat'].unique()

In [ ]:
df["Merchant Type Code"].unique()

array([5813, 6540, 5541, 6300, 4900, 4722, 7999, 6012, 5983, 4112, 5814,
       8299, 8220, 4814, 5912, 5816, 7994, 9399, 5399, 5111, 4816, 5411,
       5732, 4789, 7399, 4784, 8062, 7011, 8211, 5921, 5999, 5451, 4121,
       5441, 8099, 5311, 8999, 5651, 9311, 4812, 7278, 8071, 5262, 7392,
       7299, 5699, 7832, 5817, 5722, 5812, 5499, 5193, 5944, 4899, 6051,
       5251, 5047, 5946, 4214, 5094, 8011, 5947, 5691, 7230, 5661, 7298,
       5533, 2842, 5734, 5942, 5970, 5331, 5072,  780, 5977, 5945, 5462,
       3595, 1761, 5511, 4111, 8043, 5571, 5712, 5960, 5713, 5422, 5811,
       5964, 5139, 5948, 5621, 7372, 5611, 5309, 5641, 7296, 6513, 5137,
       8398, 5261, 5815, 5965, 5719, 8244, 5192, 5697, 5941, 5995, 7394,
       5045, 4215, 5968, 7210, 5655, 5039, 7996, 4131, 8241, 8021, 7829,
       5013, 8931, 5122, 5993, 7542, 5940, 7538, 7631, 5271, 5310, 5065,
       9211, 5735, 5992, 2741, 7221, 5211, 6211, 7523, 5733, 8734, 8699,
       9222, 7338, 7277, 5592, 7349, 7997, 5943, 55

In [ ]:
un= [5813, 6540, 5541, 6300, 4900, 4722, 7999, 6012, 5983, 4112, 5814,
       8299, 8220, 4814, 5912, 5816, 7994, 9399, 5399, 5111, 4816, 5411,
       5732, 4789, 7399, 4784, 8062, 7011, 8211, 5921, 5999, 5451, 4121,
       5441, 8099, 5311, 8999, 5651, 9311, 4812, 7278, 8071, 5262, 7392,
       7299, 5699, 7832, 5817, 5722, 5812, 5499, 5193, 5944, 4899, 6051,
       5251, 5047, 5946, 4214, 5094, 8011, 5947, 5691, 7230, 5661, 7298,
       5533, 2842, 5734, 5942, 5970, 5331, 5072,  780, 5977, 5945, 5462,
       3595, 1761, 5511, 4111, 8043, 5571, 5712, 5960, 5713, 5422, 5811,
       5964, 5139, 5948, 5621, 7372, 5611, 5309, 5641, 7296, 6513, 5137,
       8398, 5261, 5815, 5965, 5719, 8244, 5192, 5697, 5941, 5995, 7394,
       5045, 4215, 5968, 7210, 5655, 5039, 7996, 4131, 8241, 8021, 7829,
       5013, 8931, 5122, 5993, 7542, 5940, 7538, 7631, 5271, 5310, 5065,
       9211, 5735, 5992, 2741, 7221, 5211, 6211, 7523, 5733, 8734, 8699,
       9222, 7338, 7277, 5592, 7349, 7997, 5943, 5532,  742, 5631, 5521,
       1520, 7512,  763, 5231, 8111, 7699, 5200, 7361, 7993, 7911, 5969,
       5950, 3533, 7622, 5932, 7641, 7375, 3747, 8249, 7623, 5971, 7534,
       1771, 3177, 7311, 5973, 3513, 5085, 5199, 5997, 5949, 7929, 7941,
       8050, 5051, 5300, 8661, 4829, 7998, 5131, 5198, 7991, 3635, 7339,
       7531, 1740, 4511, 4582, 8641, 3020, 3659, 3562]

In [ ]:
un=list(map(str, un))

In [ ]:
#un

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
token = tokenizer.encode_plus(
    df['Merchant Location'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [ ]:
#token.input_ids

In [ ]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [ ]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['Merchant Location'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [ ]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [ ]:
labels = np.zeros((len(df), num_label))
labels.shape

(49233, 206)

In [ ]:
labels[np.arange(len(df)), df['encoded_cat'].values] = 1 # one-hot encoded target tensor

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(206,), dtype=tf.float64, name=None))>

In [ ]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [ ]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(206,), dtype=tf.float64, name=None))>

In [ ]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [ ]:
p = 0.8
train_size = int((len(df)//16)*p)

In [ ]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [ ]:
from transformers import TFBertModel

In [ ]:
model = TFBertModel.from_pretrained('bert-base-uncased') #bert-base-cased

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(num_label, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [ ]:
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2
)

Epoch 1/2
2461/2461 [==============================] - 2314s 936ms/step - loss: 2.0342 - accuracy: 0.5936 - val_loss: 1.2458 - val_accuracy: 0.7378
Epoch 2/2
2461/2461 [==============================] - 2305s 937ms/step - loss: 1.1709 - accuracy: 0.7553 - val_loss: 0.9501 - val_accuracy: 0.8016


In [ ]:
sentiment_model.save('sentiment_model')

In [ ]:
sentiment_model = tf.keras.models.load_model('sentiment_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=un):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [ ]:
input_text = input('Enter movie review here: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")

Enter movie review here: ANIMESH PETROLEUM      YAVATMAL     MHIN
Predicted Sentiment: 5462
